In [1]:
from __future__ import division
import os
import sys
import glob

In [2]:
%%file paralp.h

int ratio_test1(double *dy, int *idy,int ndy, double *y, double *ybar, double mu);
void paralp(double *obj, double *mat, double *rhs, int *m0 , int *n0, 
            double *opt, int *status, double *lambda_min, double *rhs_bar, double *obj_bar);
void solver21(int m,int n,int nz,int *ia, int *ka, double *a,double *b, double *c);


Overwriting paralp.h


In [3]:
%%file _paralp.pyx

import numpy as np
cimport numpy as np

np.import_array()

# cdefine the signature of the c function
cdef extern from "paralp.h":
    void paralp(double *obj, double *mat, double *rhs, int *m0 , 
                int *n0, double *opt, int *status, double *lambda_min, 
                double *rhs_bar, double *obj_bar)
    
def mainfunc(np.ndarray[double, ndim=1, mode="c"] obj not None,
             np.ndarray[double, ndim=2, mode="c"] mat not None,
             np.ndarray[double, ndim=1, mode="c"] rhs not None,
             np.ndarray[double, ndim=1, mode="c"] obj_bar not None,
             np.ndarray[double, ndim=1, mode="c"] rhs_bar not None,
             double lambda_min):   

    #Dimensions    
    cdef int m  = len(rhs)
    cdef int n  = len(obj)
    cdef int m0 = mat.shape[0]
    cdef int n0 = mat.shape[1]
    cdef int m1 = len(rhs_bar)
    cdef int n1 = len(obj_bar)
    
    #Define output
    cdef np.ndarray opt = np.zeros((len(obj),), dtype = np.float64, order='C')
    cdef int status = 0
    
    if (m != m0 or n != n0 or m != m1 or n != n1):
        raise ValueError("Dimensions do not match.")
    
    if ((obj_bar < 0.).any() or (rhs_bar < 0.).any()):
        raise ValueError("The pertubation vector obj_bar and rhs_bar must be nonnegative.")
        
    #Call external C function
    paralp(<double*> np.PyArray_DATA(obj),
           <double*> np.PyArray_DATA(mat),
           <double*> np.PyArray_DATA(rhs),
           &m0,
           &n0,
           <double*> np.PyArray_DATA(opt),
           &status,
           <double*> &lambda_min,
           <double*> np.PyArray_DATA(rhs_bar),
           <double*> np.PyArray_DATA(obj_bar))

    if (status == 0): 
        #print "optimal solution found! \n"
        return opt

    elif (status == 1):
        print "The problem is infeasible! \n"
    
    elif (status == 2):
        print "The problem is unbounded! \n"


Overwriting _paralp.pyx


In [4]:
%%file setup.py

from distutils.core import setup, Extension
import numpy as np
import numpy.distutils.misc_util
from Cython.Distutils import build_ext

setup(
    name = 'paralp',
    version = '1.0',
    description = 'C wrapper for paralp function',
    author = 'John Pura',
    author_email = 'john.pura@duke.edu',
    cmdclass={'build_ext': build_ext},
    ext_modules=[Extension("paralp",
                           sources=["_paralp.pyx", "paralp.c",
                                    "lu.c","tree.c","linalg.c",
                                    "heap.c"],
                 include_dirs=[np.get_include(), ])]
)


Overwriting setup.py


In [5]:
!python setup.py build_ext --inplace

running build_ext
cythoning _paralp.pyx to _paralp.c
building 'paralp' extension
C compiler: gcc -pthread -fno-strict-aliasing -g -O2 -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC

compile options: '-I/home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include -I/home/bitnami/anaconda/include/python2.7 -c'
gcc: _paralp.c
In file included from /home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1804:0,
                 from /home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:17,
                 from /home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from _paralp.c:232:
/home/bitnami/anaconda/lib/python2.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:15:2: warning: #warning "Using deprecated NumPy API, disable it by " "#defining NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy 